## Calculation of Merkle Root and Proofs for Pussy Final Distribution

In [1]:
import pandas as pd
import os
from multiprocess import Pool
from math import ceil
import json
from tqdm.notebook import tqdm
from IPython.display import display, HTML

from src.contract_utils import instantiate_contract, execute_contract_bash, query_contract, get_proofs

INITIAL_BALANCE = str(100_000_000_000)

COEF_UP = str(5)
COEF_DOWN = str(3)
TARGET_CLAIM = str(10)

NUMBER_OF_THREADS = 10
BASH_SIZE = 1000

WALLET_ADDRESS = os.getenv('WALLET_ADDRESS')
DISPLAY_TX_EXECUTION = False

INIT_SUBGRAPH_CONTRACTS = False
SUBGRAPH_CODE_ID = str(40)
NAME_SUBGRAPH_CONTRACT_ADDRESS = 'bostrom1rncw9n73gm30vhrv6e4p603hav0gue8y5y9fgqa84k4atf5pqvfqcrnpl6'
AVATAR_SUBGRAPH_CONTRACT_ADDRESS = 'bostrom164w2vl7z7lpuvex6z3ru0v55fgq3dmvxuqt0aejp49w7fyc8g6kshreggq'
PROOF_SUBGRAPH_CONTRACT_ADDRESS = 'bostrom1543j9n7slzff3curyac7ylf2ctg7rk9zjf9ehj08eqx57xj33zzqdy6ga4'

INIT_PASSPORT_CONTRACT = False
PASSPORT_CODE_ID = str(25)
PASSPORT_CONTRACT_ADDRESS = 'bostrom15hzg7eaxgs6ecn46gmu4juc9tau2w45l9cnf8n0797nmmtkdv7jscv88ra'

INIT_GIFT_CONTRACT = False
GIFT_CODE_ID = str(20)
GIFT_CONTRACT_ADDRESS = 'bostrom1rt2acjyhs4jfjdq56pftpu7762hy9gfl63je6fnhwrc5p5y4kmuqxg0262'
MERKLE_ROOT = 'd8dfb2c769cac706bb3d9cea1b681ff6ba5f680f0af05cefbe68a74027317ab0'

CALCULATE_PROOFS = True
STATE_FILE_NAME = 'data/pussygift_state_final_221228.csv'
ROOT_SOURCE_FILE_NAME = 'data/pussy_root_source_final.json'
PROOF_FILE_NAME = 'data/pussy_proof_final.json'
ROOT_FILE_NAME = 'data/pussy_root_final'

### Create source data for Merkle Tree

In [2]:
if CALCULATE_PROOFS:
    state_df = pd.read_csv(STATE_FILE_NAME)
    display(HTML(state_df.head().to_html(index=False, notebook=True, show_dimensions=False)))

    state_df = state_df.rename(columns={'gift': 'amount'})

    root_source_list = state_df[['address', 'amount']].to_dict(orient='records')
    number_of_addresses = len(root_source_list)
    print(f'Number of addresses: {number_of_addresses:>,}')

    with open(ROOT_SOURCE_FILE_NAME, 'w') as outfile:
        outfile.write(str(root_source_list).replace("'", '"'))

address,grade,segment,gift
cosmos100000aeu2lh0jrrnmn2npc88typ25u7td9wqxv,2.0,CosmosHub,1820000000000
cosmos10005vr6w230rer02rgwsvmhh0vdpk9hv6wla36,1.0,CosmosHub,210000000000
cosmos100080ak7ge6pkqf6s3el7et0jd579n65fytesw,1.0,CosmosHub,210000000000
cosmos1000e3j8mesez4pma6fpn42ufn2xluhvh59uwt4,1.0,CosmosHub,210000000000
cosmos1000g09cm584wzmwuszdyvryg6p62xkauqud2jc,1.0,CosmosHub,210000000000


Number of addresses: 1,380,697


### Calculate Merkle Root and Proofs

In [3]:
if CALCULATE_PROOFS:
    tasks = list(
        (
        (ROOT_SOURCE_FILE_NAME,
         f'pussy_final_temp/proofs_{i}.json',
         i * BASH_SIZE,
         min(number_of_addresses, (i + 1) * BASH_SIZE))
        for i in range(ceil(number_of_addresses/BASH_SIZE))
        )
    )
    print(f'First task: {tasks[0]}\nLast task: {tasks[-1]}\nTotal tasks: {len(tasks)}\nThreads: {NUMBER_OF_THREADS:>,}')
    with Pool(processes=NUMBER_OF_THREADS) as pool:
        res = pool.starmap(get_proofs, tasks, 1)
    assert res == [True] * len(res)

First task: ('data/pussy_root_source_final.json', 'pussy_final_temp/proofs_0.json', 0, 1000)
Last task: ('data/pussy_root_source_final.json', 'pussy_final_temp/proofs_1380.json', 1380000, 1380697)
Total tasks: 1381
Threads: 10


### Get proofs and root from temporary files

In [4]:
if CALCULATE_PROOFS:
    roots = []
    proofs = []
    for task in tqdm(tasks):
        with open(task[1], 'r') as proof_file:
            root_and_proof_json = json.load(proof_file)
        if len(proofs) == 0:
            proofs = root_and_proof_json['proofs']
        else:
            proofs.extend(root_and_proof_json['proofs'])
        roots.append(root_and_proof_json['merkle_root'])

    assert roots == [roots[0]] * len(roots)
    root = roots[0]
    proofs_df = pd.DataFrame(proofs)
    proofs_df.to_csv('pussy_proofs_final.csv', header=True, index=True)
else:
    root = MERKLE_ROOT
    proofs_df = pd.read_csv('pussy_proofs_final.csv')
print(f'Merkle root: {root}')
print(f'Number of proofs: {len(proofs_df):>,}')
display(HTML(proofs_df.head().to_html(index=False, notebook=True, show_dimensions=False)))

  0%|          | 0/1381 [00:00<?, ?it/s]

Merkle root: ed39bd35cd17b492838ca2c82196cb55d19e9b02154bf9ce6e3a30c59ea1ec7f
Number of proofs: 1,380,697


address,amount,proof
cosmos100000aeu2lh0jrrnmn2npc88typ25u7td9wqxv,1830000000000,[1f4d76ff0fd98a94f78d8b57d8c45656dffaa2f9c69086...
cosmos10005vr6w230rer02rgwsvmhh0vdpk9hv6wla36,210000000000,[d194bea65a73dc5b7fcf8b11bf79678ac868f328dfd1f8...
cosmos100080ak7ge6pkqf6s3el7et0jd579n65fytesw,210000000000,[a237820e75ddbfa9ddf39ecd148ee7d408e8b383f9a8a3...
cosmos1000e3j8mesez4pma6fpn42ufn2xluhvh59uwt4,210000000000,[4f9d777f744de0715f0fcca0de9c6e1b6870f4ea536067...
cosmos1000g09cm584wzmwuszdyvryg6p62xkauqud2jc,210000000000,[8a3a025ee739a561f90d11b033eb07124b3d27ee8a1a85...


## Instantiate Contracts
### Instantiate SUBGRAPH Contracts

In [ ]:
if INIT_SUBGRAPH_CONTRACTS:
    name_subgraph_contract_address = \
        instantiate_contract(
            init_query=f'''{{"owner":"{WALLET_ADDRESS}", "executer":"{WALLET_ADDRESS}"}}''',
            contract_code_id=SUBGRAPH_CODE_ID,
            contract_label='test name subgraph')
    avatar_subgraph_contract_address = \
        instantiate_contract(
            init_query=f'''{{"owner":"{WALLET_ADDRESS}", "executer":"{WALLET_ADDRESS}"}}''',
            contract_code_id=SUBGRAPH_CODE_ID,
            contract_label='test avatar subgraph')
    proof_subgraph_contract_address = \
        instantiate_contract(
            init_query=f'''{{"owner":"{WALLET_ADDRESS}", "executer":"{WALLET_ADDRESS}"}}''',
            contract_code_id=SUBGRAPH_CODE_ID,
            contract_label='test proof subgraph')
else:
    name_subgraph_contract_address = NAME_SUBGRAPH_CONTRACT_ADDRESS
    avatar_subgraph_contract_address = AVATAR_SUBGRAPH_CONTRACT_ADDRESS
    proof_subgraph_contract_address = PROOF_SUBGRAPH_CONTRACT_ADDRESS
print(f'Name subgraph contract address: {name_subgraph_contract_address}\n'
      f'Avatar subgraph contract address: {avatar_subgraph_contract_address}\n'
      f'Proof subgraph contract address: {proof_subgraph_contract_address}')

### Instantiate Passport Contract

In [ ]:
if INIT_PASSPORT_CONTRACT:
    passport_contract_address = \
        instantiate_contract(
            init_query=f'''{{"name":"CPT", "minter":"{WALLET_ADDRESS}", "owner":"{WALLET_ADDRESS}", "symbol":"CPT", "avatar_subgraph": "{avatar_subgraph_contract_address}", "name_subgraph": "{name_subgraph_contract_address}", "proof_subgraph": "{proof_subgraph_contract_address}"}}''',
            contract_code_id=PASSPORT_CODE_ID,
            contract_label='test passport')
else:
    passport_contract_address = PASSPORT_CONTRACT_ADDRESS
print(f'Passport contract address: {passport_contract_address}')

### Set executor in the Subgraph Contracts

In [ ]:
def set_executor_subgraph(subgraph_contract_address: str, new_executor_address: str, display_data: bool = False):
    return execute_contract_bash(execute_query=f'''{{"update_executer":{{"new_executer":"{new_executor_address}"}}}}''',
                            contract_address=subgraph_contract_address,
                            gas=600000,
                            display_data=display_data)

if INIT_PASSPORT_CONTRACT or INIT_SUBGRAPH_CONTRACTS:
    set_executor_subgraph(subgraph_contract_address=name_subgraph_contract_address, new_executor_address=passport_contract_address)
    set_executor_subgraph(subgraph_contract_address=avatar_subgraph_contract_address, new_executor_address=passport_contract_address)
    set_executor_subgraph(subgraph_contract_address=proof_subgraph_contract_address, new_executor_address=passport_contract_address)

### Instantiate Gift Contract

In [ ]:
if INIT_GIFT_CONTRACT:
    gift_contract_address = \
        instantiate_contract(
            init_query=f'''{{"owner":"{WALLET_ADDRESS}", "passport":"{passport_contract_address}", "allowed_native":"boot", "initial_balance":"{INITIAL_BALANCE}", "coefficient_up":"{COEF_UP}", "coefficient_down":"{COEF_DOWN}", "coefficient":"{COEF_UP}", "target_claim":"{TARGET_CLAIM}"}}''',
            contract_code_id=GIFT_CODE_ID,
            amount=INITIAL_BALANCE,
            contract_label='test gift')
else:
    gift_contract_address = GIFT_CONTRACT_ADDRESS
print(f'Gift contract address: {gift_contract_address}')

## Register Merkle Root

In [ ]:
root_register_output = execute_contract_bash(execute_query=f'''{{"register_merkle_root":{{"merkle_root":"{root}"}}}}''',
                                        contract_address=gift_contract_address)

In [ ]:
print(f'Gift contract {gift_contract_address}')
print(query_contract(query='''{"merkle_root": {}}''',
                     contract_address=gift_contract_address))
